# Eigenmode simulation of coupled qubits

In [ ]:
%load_ext autoreload
%autoreload 2

## Rendering the design

In [ ]:
from chip_initialization import create_chip_and_gui

design, gui = create_chip_and_gui()

from design import render_qiskit_metal_design

render_qiskit_metal_design(design, gui)

## Creating the study and optimization targets

In [ ]:
import names as n
import mini_studies as ms
import optimization_targets as ot

In [ ]:
MINI_STUDY_GROUP = [n.NBR_1, n.NBR_2]
MINI_STUDY = ms.get_mini_study_2qb_resonator_coupler()
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=MINI_STUDY_GROUP,
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=True,
    opt_target_resonator_qubit_chi=True,
    opt_target_coupler_freq=True,
    use_simple_resonator_qubit_chi_relation=True,
)

## Creating design analysis objects

In [ ]:
import time
import numpy as np
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qdesignoptimizer.utils.utils import get_save_path

# Closing Ansys hfss
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)

design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=get_save_path("out/", n.CHIP_NAME),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_TWO_QB,
)

## Optimization step

In [ ]:
# number of runs of optimization and number of passes for simulation at each run
group_runs = 10
group_passes_start = 8
group_passes_stop = 15
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(
        min(group_passes_start + i, group_passes_stop)
    )  # successive increase of number of passes for faster convergence [8, 9, 10, 11, 12, 13, 14, 15, 15, 15]
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

## Results

### Eigenmodes

In [ ]:
design_analysis.get_eigenmode_results()

### Cross Kerr

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

## Update parameters

In [ ]:
design_analysis.overwrite_parameters()

## Close

In [ ]:
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()